#Model2: Predict Game’s Results Based on Oliver’s Four Factors 

7 Models to predict the accuracy of Four Factors in influencing the winning rate of the game


In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
lin_regression = LinearRegression()

from sklearn.metrics import mean_squared_error, r2_score

In [52]:
from sklearn.linear_model import LogisticRegression
log_regression = LogisticRegression(solver='lbfgs')

In [53]:
data = pd.read_csv('/Users/wenhui/Desktop/Python project/2012-18_teamBoxScore(1).csv')
data.head()
data.columns

Index(['gmDate', 'gmTime', 'seasTyp', 'offLNm1', 'offFNm1', 'offLNm2',
       'offFNm2', 'offLNm3', 'offFNm3', 'teamAbbr',
       ...
       'opptFIC40', 'opptOrtg', 'opptDrtg', 'opptEDiff', 'opptPlay%', 'opptAR',
       'opptAST/TO', 'opptSTL/TO', 'poss', 'pace'],
      dtype='object', length=121)

In [54]:
#Calculate the team's and opponent's Effective Field Goal Percentage as given in the dataset:
#Formula: team's eFG%=(teamFGM+0.5*team3PM)/teamFGA  opponent's eFG%=(opptFGM+0.5*oppt3PM)/opptFGA
#_________________
#Calculate the team's and opponent's Turnover Percentage(TOV%) as not given in the dataset:
#Formula: teamTOV%=teamTO / (teamFGA + 0.44*teamFTA + teamTO) opptTOV%=opptTO/(opptFGA+0.44*opptFTA+opptTO)         
data['teamTOV%'] = data['teamTO'] / (data['teamFGA'] + 0.44*data['teamFTA']+data['teamTO'])
data['opptTOV%']= data['opptTO'] / (data['opptFGA'] + 0.44*data ['opptFTA']+data['opptTO'])
#__________________
#Calculate the team's Offencive and Defensive Rebound Percentage(OREB% & DREB%) as not given in the dataset:
#Formula: teamOREB%= teamORB / (teamORB + opptDRB)  teamDREB%= teamDRB / (opptORB + teamDRB)
data['teamOREB%']=data['teamORB']/ (data['teamORB']+data['opptDRB'])
data['teamDREB%']=data['teamDRB']/ (data['opptORB']+data['teamDRB'])
#__________________ 
#Calculate the team's and opponent's Freethrow Rate(teamFTR% & opptFTR%) as not given in the dataset:
#Formula: teamFTR%= teamFTA / teamFGA    opptFTR%= opptFTA / opptFGA
data['teamFTR%']=data['teamFTA'] / data['teamFGA']
data['opptFTR%']=data['opptFTA'] / data['opptFGA']

In [55]:
data = data[['teamRslt','teamEFG%','opptEFG%','teamTOV%','opptTOV%','teamOREB%','teamDREB%','teamFTR%','opptFTR%']]
data.head(2).append(data.tail(2))

,teamRslt,teamEFG%,opptEFG%,teamTOV%,opptTOV%,teamOREB%,teamDREB%,teamFTR%,opptFTR%
0,Loss,0.4000,0.5000,0.116279,0.191466,0.333333,0.538462,0.222222,0.278481
1,Win,0.5000,0.4000,0.191466,0.116279,0.461538,0.666667,0.278481,0.222222
14756,Loss,0.4573,0.5188,0.138231,0.110220,0.160000,0.842105,0.146341,0.250000
14757,Win,0.5188,0.4573,0.110220,0.138231,0.157895,0.840000,0.250000,0.146341


In [56]:
#查看是否存在缺失值
data.isnull().sum()

teamRslt     0
teamEFG%     0
opptEFG%     0
teamTOV%     0
opptTOV%     0
teamOREB%    0
teamDREB%    0
teamFTR%     0
opptFTR%     0
dtype: int64

In [57]:
#将文本数据转化为分类变量
data['target'] = data['teamRslt'].map(lambda x:1 if x == 'Win' else 0)
data.head()

,teamRslt,teamEFG%,opptEFG%,teamTOV%,opptTOV%,teamOREB%,teamDREB%,teamFTR%,opptFTR%,target
0,Loss,0.4000,0.5000,0.116279,0.191466,0.333333,0.538462,0.222222,0.278481,0
1,Win,0.5000,0.4000,0.191466,0.116279,0.461538,0.666667,0.278481,0.222222,1
2,Loss,0.5600,0.5949,0.154859,0.079145,0.184211,0.871795,0.373333,0.405063,0
3,Win,0.5949,0.5600,0.079145,0.154859,0.128205,0.815789,0.405063,0.373333,1
4,Win,0.5000,0.5130,0.114373,0.133792,0.225000,0.673913,0.211765,0.402597,1


In [58]:

col=['teamEFG%','opptEFG%','teamTOV%','opptTOV%','teamOREB%','teamDREB%','teamFTR%','opptFTR%']
X=data[col]
y =data['target']
X.tail()
X.shape
y.shape

(14758,)

In [59]:
# 拆分训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
y_test.shape 

(4428,)

In [60]:
# Logistic Regression Model

logreg = LogisticRegression()
%time logreg.fit(X_train, y_train.values.reshape(-1))
y_pred = logreg.predict(X_test)
logreg.score(X_test,y_test)

CPU times: user 106 ms, sys: 38.4 ms, total: 145 ms
Wall time: 72.7 ms


0.9311201445347786

In [61]:
# Support Vector Classifier Model

from sklearn.svm import SVC
from sklearn import svm
clf = SVC(gamma='scale',probability=True)
%time clf.fit(X_train, y_train)
clf.score(X_test,y_test)

CPU times: user 4.55 s, sys: 283 ms, total: 4.84 s
Wall time: 4.86 s


0.9451219512195121

In [62]:
# Linear Support Vector Classifier Model

from sklearn.svm import LinearSVC
lclf = LinearSVC(random_state=0, tol=1e-5, max_iter=1000)
%time lclf.fit(X_train, y_train)
lclf.score(X_test,y_test)

CPU times: user 53.1 ms, sys: 3.82 ms, total: 56.9 ms
Wall time: 56.8 ms


0.9444444444444444

In [63]:
# BaggingSVC Ensemble classification Model
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
advclf = BaggingClassifier(base_estimator=SVC(gamma='scale'), n_estimators=10, random_state=0)
%time advclf.fit(X_train, y_train)
advclf.score(X_test,y_test)

CPU times: user 3.65 s, sys: 144 ms, total: 3.8 s
Wall time: 3.28 s


0.9453477868112015

In [64]:
# Random Forest Classifier Model
from sklearn.ensemble import RandomForestClassifier
rdf = RandomForestClassifier(max_depth=8, random_state=0, n_estimators=300)
%time rdf.fit(X_train, y_train)
rdf.score(X_test,y_test)

CPU times: user 2.67 s, sys: 24 ms, total: 2.7 s
Wall time: 2.71 s


0.8970189701897019

In [65]:
# KNeighbors Classifier Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
knn = KNeighborsClassifier(n_neighbors=10)
%time knn.fit(X_train, y_train)
knn.score(X_test, y_test)

CPU times: user 9.34 ms, sys: 2.44 ms, total: 11.8 ms
Wall time: 12.2 ms


0.8873080397470642

In [66]:
# XGBoost Model
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgbModel = XGBClassifier()
%time xgbModel.fit(X_train, y_train)
xgbPredictions = xgbModel.predict(X_test)
xgbAccuracy = accuracy_score(y_test, xgbPredictions)
print(xgbAccuracy)

CPU times: user 2.16 s, sys: 225 ms, total: 2.38 s
Wall time: 339 ms
0.9295392953929539
